# EDA

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
#df.select_dtypes(include='number')

## testing on QBs

In [66]:
fdat = pd.read_csv("Madden/ffscores.csv", index_col=0)
fdat['game'] = fdat.apply(lambda x: x['team'].strip() + str(x['weekn']), axis=1)
fdat.head()

,Player,Opp,PPR PTS,0.5 PPR,STD PTS,C/A,PCT,YDS,TD,INT,...,YDS.2,AVG.1,TD.2,2PC,FUM,Name,pos,team,weekn,game
0,"Marcedes Lewis (TE, CHI)",@WAS,0.00,0.00,0.00,0/0,0%,0,0,0,...,0,0.0,0,0,0,Marcedes Lewis,TE,chi,5,chi5
1,"Josh Johnson (QB, BAL)",@PIT,0.00,0.00,0.00,0/0,0%,0,0,0,...,0,0.0,0,0,0,Josh Johnson,QB,bal,5,bal5
2,"Matthew Slater (WR, NE)",NO,0.00,0.00,0.00,0/0,0%,0,0,0,...,0,0.0,0,0,0,Matthew Slater,WR,ne,5,ne5
3,"Brian Hoyer (QB, LV)",GB,0.00,0.00,0.00,0/0,0%,0,0,0,...,0,0.0,0,0,0,Brian Hoyer,QB,lv,5,lv5
4,"Matthew Stafford (QB, LAR)",PHI,16.88,20.88,20.88,21/37,56.8%,222,2,0,...,0,0.0,0,0,0,Matthew Stafford,QB,lar,5,lar5


In [67]:
fdat_qb = fdat.loc[fdat['pos']=="QB",:]
fdat_qb = fdat_qb.groupby('game').agg(sum)
fdat_qb

/var/folders/qz/80bqrsy90pj24jdm4b1q067czx01fz/T/ipykernel_10746/1352065986.py:2: FutureWarning: The operation <built-in function sum> failed on a column. If any error is raised, this will raise an exception in a future version of pandas. Drop these columns to avoid this warning.
  fdat_qb = fdat_qb.groupby('game').agg(sum)


,PPR PTS,0.5 PPR,STD PTS,YDS,TD,INT,RUSH,YDS.1,AVG,TD.1,REC,TAR,YDS.2,AVG.1,TD.2,2PC,FUM,weekn
game,,,,,,,,,,,,,,,,,,
ari1,-0.02,-0.02,-0.02,132,0,0,3,-3,-1.0,0,0,0,0,0.0,0,0,3,2
ari2,-0.02,-0.02,-0.02,132,0,0,3,-3,-1.0,0,0,0,0,0.0,0,0,3,4
ari3,17.06,19.06,19.06,189,1,0,6,55,9.2,0,0,0,0,0.0,0,0,0,6
ari4,-0.02,-0.02,-0.02,132,0,0,3,-3,-1.0,0,0,0,0,0.0,0,0,3,8
ari5,10.74,14.74,14.74,166,2,2,3,1,0.3,0,0,0,0,0.0,0,0,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
was1,16.18,18.18,18.18,202,1,1,2,11,5.5,1,0,0,0,0.0,0,0,1,2
was2,16.18,18.18,18.18,202,1,1,2,11,5.5,1,0,0,0,0.0,0,0,1,4
was3,4.60,4.60,4.60,170,0,4,1,18,18.0,0,0,0,0,0.0,0,0,0,6


In [58]:
fdat_qb.index

Index(['ari1', 'ari2', 'ari3', 'ari4', 'ari5', 'atl1', 'atl2', 'atl3', 'atl4',
       'atl5',
       ...
       'ten1', 'ten2', 'ten3', 'ten4', 'ten5', 'was1', 'was2', 'was3', 'was4',
       'was5'],
      dtype='object', name='game', length=156)

In [6]:
ddat = pd.read_csv("Madden/defense.csv", index_col=0)
ddat.head()

,Total,Sack,Int,PD,FF,team,opp,weekn,home,points,...,pass_Att,pass_Yds,pass_TD,rush_Rush,rush_Yds,rush_Avg,rush_TD,rec_Rec,rec_Yds,rec_Avg
0,58.0,2.0,3.0,4.0,0.0,lv,gb,5,1,13,...,30,182,0,25.0,110.0,18.3,1.0,16.0,182.0,90.5
1,71.0,4.0,1.0,4.0,0.0,gb,lv,5,0,17,...,31,208,1,29.0,96.0,26.0,1.0,22.0,208.0,69.0
2,60.0,3.0,0.0,4.0,1.0,was,chi,5,1,40,...,29,282,4,32.0,178.0,22.4,0.0,15.0,282.0,42.2
3,73.0,5.0,1.0,5.0,1.0,chi,was,5,0,20,...,51,388,2,10.0,29.0,6.5,0.0,37.0,388.0,116.0
4,68.0,4.0,1.0,2.0,3.0,den,nyj,5,1,31,...,26,199,0,32.0,234.0,28.5,1.0,19.0,199.0,64.2


In [47]:
team_dat = []
for t in ddat.opp.unique():
    dat1 = ddat.loc[ddat['opp']==t,:].sort_values("weekn")
    opp_def = {}
    for opp in dat1.team.unique():
        df = ddat.loc[ddat['team']==opp,:].select_dtypes(include="number")
        opp_def[opp] = df.mean()
    opp_df = pd.DataFrame(opp_def)
    dat1_df = dat1.select_dtypes(include="number").T
    diff_df = pd.DataFrame(dat1_df.to_numpy() - opp_df.to_numpy(), 
                       index=dat1_df.index, 
                       columns=[t+s for s in np.arange(1,dat1.weekn.max()+1).astype(str)])
    team_dat.append(diff_df.T)
all_def_df = pd.concat(team_dat)

In [60]:
all_def_df = all_def_df.loc[fdat_qb.index,:]
all_def_df

,Total,Sack,Int,PD,FF,weekn,home,points,pass_Comp,pass_Att,pass_Yds,pass_TD,rush_Rush,rush_Yds,rush_Avg,rush_TD,rec_Rec,rec_Yds,rec_Avg
game,,,,,,,,,,,,,,,,,,,
ari1,0.2,-0.2,-0.4,-1.4,1.0,-2.0,0.4,-16.0,1.2,-2.0,-119.8,-2.0,-3.0,-37.6,12.58,-0.8,1.2,-119.8,-28.90
ari2,-2.0,-1.0,-0.4,-2.0,-0.6,-1.0,-0.4,-2.6,1.8,1.2,-4.4,-0.2,0.2,-0.4,-0.94,0.2,1.8,-4.4,9.34
ari3,2.2,-1.0,-1.4,-4.2,-1.2,0.0,-0.4,11.4,1.0,-5.6,3.2,-0.2,2.4,98.6,17.92,1.2,1.0,3.2,-3.12
ari4,8.8,-1.6,-1.6,-4.0,-0.2,1.0,0.4,2.4,1.8,0.8,44.6,1.0,7.6,40.8,-1.48,-0.4,1.8,44.6,-6.20
ari5,-12.8,0.0,0.8,3.6,0.4,2.0,-0.4,-2.8,-3.4,1.4,-47.6,0.4,-7.0,-12.0,2.50,0.2,-3.4,-47.6,-6.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
was1,-1.4,3.0,0.2,2.2,1.4,-2.0,-0.6,-7.2,-6.2,-4.0,-72.4,-0.6,-1.8,-32.2,2.22,-0.4,-6.2,-72.4,2.76
was2,-5.8,1.6,-0.6,1.0,0.0,-1.0,0.4,-1.2,3.2,8.2,17.2,-0.6,-8.6,-65.6,4.40,0.4,3.2,17.2,15.80
was3,-7.4,4.8,2.4,0.2,-0.6,0.0,-0.6,-13.0,-0.8,-0.4,-48.4,-0.8,-10.0,-29.0,0.46,-0.8,-0.8,-48.4,-12.76


## Making some models
todo:
- How to incorporate k-fold validation?
- Gradient boosting?

Try:
- SVR
    - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
    - https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_kernel_ridge_regression.html#sphx-glr-auto-examples-miscellaneous-plot-kernel-ridge-regression-py
- LASSO
- ?

In [ ]:
svr = GridSearchCV(
    SVR(kernel="rbf", gamma=0.1),
    param_grid={"C": [1e0, 1e1, 1e2, 1e3], "gamma": np.logspace(-2, 2, 5)},
)

In [63]:
# scaler = StandardScaler()
# scaler.fit_transform(all_def_df)

In [68]:
svr = SGDRegressor(max_iter=1000, tol=1e-3)
svr.fit(all_def_df, fdat_qb.loc[:,"PPR PTS"])

SGDRegressor()

In [70]:
svr.coef_

array([ 2.49180947e+10, -1.25857152e+11,  1.20341531e+10, -5.25419528e+10,
       -2.82981646e+10, -1.02135754e+10,  3.48043445e+09,  3.34572304e+10,
        1.07194207e+10,  3.49003361e+10, -3.16129454e+10,  6.24984953e+10,
        4.98903197e+10,  1.08502767e+11, -1.73757993e+11, -4.79277526e+10,
        1.07194207e+10, -3.16129454e+10, -5.51535668e+10])